In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

# **Imports**

In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

In [3]:
sns.set_style('whitegrid')
sns.set(rc={'figure.figsize':(15, 5)})

# **Cargamos el dataset**

In [4]:
filename = "dataset_inf_telec.csv"

In [5]:
df = pd.read_csv(
    filename,
    dtype={
        'ID_EQUIPO': int,
        'PUNTO_MEDICION': str,
        'CAPACIDAD_MAXIMA_GBS': float,
        'PASO': int,
        'LATENCIA_MS': float,
        'PORCENTAJE_PACK_LOSS': float,
        'INBOUND_GBPS': np.float64,
        'OUTBOUND_GBPS': np.float64,
    },
    parse_dates=[
        'FECHA_INICIO_MEDICION',
        'FECHA_FIN_MEDICION',
        'FECHA_HORA',
    ],
)

## **Exploremos un poco el Dataset**

In [6]:
# Veamos el tamaño del dataset
print(f"Shape = {df.shape}")

Shape = (16128, 11)


Tenemos en total:
* +16K registros
* 11 columnas (features)

In [7]:
# Veamos una muestra de 10 registros del dataset
df.sample(10)

,ID_EQUIPO,PUNTO_MEDICION,CAPACIDAD_MAXIMA_GBS,FECHA_INICIO_MEDICION,FECHA_HORA,FECHA_FIN_MEDICION,PASO,LATENCIA_MS,PORCENTAJE_PACK_LOSS,INBOUND_GBPS,OUTBOUND_GBPS
5475,62,NOC - 6720HI to BAZ,20.0,2020-05-28 14:00:00,2020-05-30 16:00:00,2020-06-04 14:00:00,1800,1.627253,0.000000,1.350094e+10,1.369058e+11
5260,28,JM - Totoral Vieja,10.0,2020-05-28 14:00:00,2020-06-02 04:30:00,2020-06-04 14:00:00,1800,2.782947,0.770902,1.153526e+09,4.768196e+09
5477,62,NOC - 6720HI to BAZ,20.0,2020-05-28 14:00:00,2020-05-30 17:00:00,2020-06-04 14:00:00,1800,1.579915,0.000000,1.333782e+10,1.339084e+11
1776,30,Carlos Paz - La Falda,1.0,2020-05-28 13:30:00,2020-05-30 14:00:00,2020-06-04 13:30:00,1800,3.697806,0.000000,5.963986e+08,5.924080e+09
15796,23,Yocsina - Mogote,10.0,2020-05-28 15:00:00,2020-05-28 17:30:00,2020-06-04 15:00:00,1800,2.059392,0.000000,2.721356e+09,4.658026e+09
5740,62,NOC - 6720HI to EDC,40.0,2020-05-28 14:00:00,2020-05-29 04:30:00,2020-06-04 14:00:00,1800,1.895442,0.000000,1.642044e+10,6.242698e+10
3421,24,EDC - NOR,20.0,2020-05-28 13:30:00,2020-05-29 20:30:00,2020-06-04 13:30:00,1800,1.782653,0.000000,5.506033e+09,6.357781e+10
7321,62,NOC - 6720HI to RPrivado,1.0,2020-05-28 14:00:00,2020-06-03 03:00:00,2020-06-04 14:00:00,1800,0.383295,0.000000,1.455444e+07,1.414752e+07
4993,28,JM - Totoral Nueva,1.0,2020-05-28 13:30:00,2020-06-03 14:30:00,2020-06-04 13:30:00,1800,1.730828,0.000000,2.173999e+04,3.276943e+05
4847,28,JM - Totoral Nueva,1.0,2020-05-28 13:30:00,2020-05-31 13:30:00,2020-06-04 13:30:00,1800,1.740122,0.000000,2.221519e+04,3.708204e+05


In [8]:
# Veamos los tipos de las columnas
df.dtypes

ID_EQUIPO                         int64
PUNTO_MEDICION                   object
CAPACIDAD_MAXIMA_GBS            float64
FECHA_INICIO_MEDICION    datetime64[ns]
FECHA_HORA               datetime64[ns]
FECHA_FIN_MEDICION       datetime64[ns]
PASO                              int64
LATENCIA_MS                     float64
PORCENTAJE_PACK_LOSS            float64
INBOUND_GBPS                    float64
OUTBOUND_GBPS                   float64
dtype: object

#### Las columnas representan:
* **ID_EQUIPO**:             Switch/Router
* **PUNTO_MEDICION**:        Línea Troncal
* **CAPACIDAD_MAXIMA_GBS**:  Capacidad maxima [Gigabit/seg]
* **FECHA_INICIO_MEDICION**: Fecha y hora inicio medicion
* **FECHA_HORA**:            Fecha y hora efectiva de la medida
* **FECHA_FIN_MEDICION**:    Fecha y hora fin de medicion
* **PASO**:
* **LATENCIA_MS**:           Latencia (tiempo de espera de la respuesta) [mseg]
* **PORCENTAJE_PACK_LOSS**:  % de paquetes perdidos (pings configurados en diferentes momentos del dia) [0 a 100]
* **INBOUND_GBPS**:          Trafico de entrada [Gigabit/seg]
* **OUTBOUND_GBPS**:         Trafico de salida  [Gigabit/seg]

In [9]:
# Veamos una descripcion general de las variables numericas del Dataset
df.describe().T

,count,mean,std,min,25%,50%,75%,max
ID_EQUIPO,16128.0,2.587500e+01,1.708320e+01,4.000000,1.100000e+01,2.400000e+01,3.100000e+01,6.200000e+01
CAPACIDAD_MAXIMA_GBS,16128.0,6.211654e+00,8.264077e+00,0.027263,1.000000e+00,1.000000e+00,1.000000e+01,4.000000e+01
PASO,16128.0,1.800000e+03,0.000000e+00,1800.000000,1.800000e+03,1.800000e+03,1.800000e+03,1.800000e+03
LATENCIA_MS,16080.0,2.775573e+00,2.086607e+00,0.250389,1.270442e+00,1.993651e+00,3.455664e+00,2.596465e+01
PORCENTAJE_PACK_LOSS,16079.0,3.834813e-01,2.800800e+00,0.000000,0.000000e+00,0.000000e+00,2.015795e-01,5.828972e+01
INBOUND_GBPS,16080.0,6.638273e+09,2.415034e+10,952.354004,1.333419e+07,1.462419e+08,2.328255e+09,2.405104e+11
OUTBOUND_GBPS,16080.0,1.368882e+10,3.197588e+10,47545.992188,2.939275e+07,8.192615e+08,9.830040e+09,2.369527e+11


In [10]:
# Veamos una descripcion general de las variables categoricas del Dataset
df.describe(include='O').T

,count,unique,top,freq
PUNTO_MEDICION,16128,48,EDC - MOP,336


## **Importante**

#### Cada `ID_EQUIPO` puede tener asociado uno o más `PUNTO_MEDICION`

In [11]:
# Veamos la cantidad de registros por ID_EQUIPO.
df.ID_EQUIPO.value_counts()

4     3024
24    2688
31    2352
62    2016
11    1680
23    1344
30    1008
28     672
41     672
25     672
Name: ID_EQUIPO, dtype: int64

In [12]:
# Veamos la cantidad de registros por PUNTO_MEDICION.
df.PUNTO_MEDICION.value_counts()

EDC - MOP                         336
BAZ - Yocsina                     336
NOC - 6720HI to N20-1             336
NOC - UTN                         336
RDB - JM                          336
SF - La Francia                   336
NOC - 6720HI to EDC               336
Yocsina - Mogote                  336
NOC - ACHALA - Servicios          336
EDC - Transporte                  336
NOC - Almacenes                   336
Yocsina - Carlos Paz              336
RDB - Escuela de Capacitacion     336
RDB - GZU                         336
NOC - 6720HI to RPrivado          336
JM - Totoral Nueva                336
EDC - RDB                         336
RDB - ET Don Bosco - San Roque    336
SF - SF Adm                       336
EDC - ET Oeste                    336
SF - SF Cliente                   336
ABA - Abasto Cliente              336
JM - Totoral Vieja                336
NOC - 6720HI to ETC               336
Carlos Paz - La Falda             336
Carlos Paz - Cosquin              336
EDC - Capita

In [28]:
for punto_medicion in list(df.PUNTO_MEDICION.unique()):
    print(punto_medicion)
    start_measurings = df[df.PUNTO_MEDICION == punto_medicion].FECHA_INICIO_MEDICION.unique()
    end_measurings = df[df.PUNTO_MEDICION == punto_medicion].FECHA_FIN_MEDICION.unique()
    print(f"\t Inicio de Medicion        : {start_measurings}")
    print(f"\t Fin de Medicion           : {end_measurings}")

    tdelta = end_measurings[0] - start_measurings[0]
    days = tdelta.astype('timedelta64[D]')
    print(f"\t Duracion de la Medicion   : {days / np.timedelta64(1, 'D')} dias")
    print("=" * 20)

ABA - Abasto Cliente
	 Inicio de Medicion        : ['2020-05-28T13:00:00.000000000']
	 Fin de Medicion           : ['2020-06-04T13:00:00.000000000']
	 Duracion de la Medicion   : 7.0 dias
ABA - Temple
	 Inicio de Medicion        : ['2020-05-28T13:30:00.000000000']
	 Fin de Medicion           : ['2020-06-04T13:30:00.000000000']
	 Duracion de la Medicion   : 7.0 dias
BAZ - Carlos Paz
	 Inicio de Medicion        : ['2020-05-28T13:30:00.000000000']
	 Fin de Medicion           : ['2020-06-04T13:30:00.000000000']
	 Duracion de la Medicion   : 7.0 dias
BAZ - Yocsina
	 Inicio de Medicion        : ['2020-05-28T13:30:00.000000000']
	 Fin de Medicion           : ['2020-06-04T13:30:00.000000000']
	 Duracion de la Medicion   : 7.0 dias
Carlos Paz - Cosquin
	 Inicio de Medicion        : ['2020-05-28T13:30:00.000000000']
	 Fin de Medicion           : ['2020-06-04T13:30:00.000000000']
	 Duracion de la Medicion   : 7.0 dias
Carlos Paz - La Falda
	 Inicio de Medicion        : ['2020-05-28T13:30:00.00000

### Notar que la duracion de la medicion por cada `PUNTO_MEDICION` es de 1 semana